In [177]:
with open('day18_inputs.txt') as f:
    content = [l.strip('\n').split(' ') for l in f]

for l in content:
    print(l)

['(4', '+', '(2', '*', '7)', '*', '4)', '+', '(6', '*', '9', '+', '8', '*', '4', '+', '7', '*', '3)', '*', '3', '*', '5']
['5', '*', '9', '+', '(5', '*', '9)', '*', '(6', '+', '2)', '+', '3', '+', '7']
['8', '+', '5', '*', '9', '*', '9', '+', '9', '+', '5']
['(7', '+', '9', '*', '8', '+', '4', '*', '6)', '+', '6', '*', '((9', '+', '9', '+', '5', '*', '7', '+', '4)', '*', '8', '*', '2', '+', '5', '*', '6', '+', '2)', '+', '2', '*', '6']
['(6', '*', '(5', '+', '8', '*', '7', '*', '8', '+', '4)', '*', '(7', '+', '7', '*', '3', '*', '5))', '+', '5', '*', '(8', '+', '(8', '+', '3', '+', '5', '+', '5)', '+', '(3', '+', '2', '+', '7', '*', '2', '*', '9)', '+', '6', '*', '5', '+', '(2', '*', '6))', '*', '((4', '*', '3)', '+', '3)', '+', '9', '*', '(3', '+', '6', '*', '2', '+', '3', '*', '8)']
['2', '*', '9', '+', '(6', '*', '2', '+', '6', '+', '(9', '+', '6', '+', '8', '+', '5', '+', '5)', '+', '(8', '+', '7', '*', '4', '+', '3)', '+', '8)', '*', '(4', '+', '4', '*', '2', '+', '6)']
['5', '+',

### Part 1 

Evaluate the expressions:
- Parentheses must be evaluated first
- BODMAS doesn't apply, precedence of calculations is just from left to right

e.g. with BODMAS, 5 + 3 * 2 would = 11, but in this case it would equal 16

#### First we get the nesting of the calculation

In [7]:
class T:
    def __init__(self, v):
        self.v = v
    def __add__(self, other):
        return T(self.v + other.v)
    def __sub__(self, other):
        return T(self.v * other.v)
    def __mul__(self, other):
        return T(self.v + other.v)

def main():
    part2 = False
    with open("day18_inputs.txt") as f:
        inp = f.read()

    lines = ['1 + 2 * 3', '1 * 2 + 3'] #[line for line in inp.split("\n") if line]
    t = 0
    for line in lines:
        for d in range(10):
            line = line.replace(f"{d}", f"T({d})")
        line = line.replace("*", "-")
        print(line)
        if part2:
            line = line.replace("+", "*")
        t += eval(line, {"T": T}).v
        print(t)
    print(t)
    
main()

T(1) + T(2) - T(3)
9
T(1) - T(2) + T(3)
14
14


In [8]:
eval('T(1) - T(2) + T(3)', {"T": T}).v

5

Brackets, Mult, Add, Sub
Brackets, Add, Mult

In [178]:
import copy

def get_nesting(calc_str):
    # Takes calc_str and returns a list of how many layers nested an item is in the string
    
    list_nesting = []
    nesting = 0
    for c in calc_str:
        # Nesting increases for a digit when it's inside open bracket(s)
        nesting += c.count('(')
        list_nesting.append(nesting)
        
        # Nesting decreases for the NEXT digit if the current digit has closed bracket(s)
        nesting -= c.count(')')
    
    return list_nesting


def operator_by_order(calc_list):
    # Given a calc_list, takes the first 3 elements and applies the middle operator
    # Returns an int
    # eg. ['3','+','5','*','6'] would be parsed as 3+5 and return 8
    a, op, b = calc_list[:3]
    return eval('{}{}{}'.format(a,op,b))

results = 0
for i in range(len(content)):
    
    # Assign calculation string
    calc_str = content[i]
    
    # Get nesting at each position, then strip ( ) from the list
    calc_str_nesting = get_nesting(calc_str)
    calc_str_clean = [n.replace('(','').replace(')','') for n in calc_str]

    # Then, join each element of the calculation, it's nesting, and it's position
    calc_str_w_nesting_and_pos = list(zip(calc_str_clean, 
                                      calc_str_nesting, 
                                      [i for i in range(len(calc_str_clean))]))
    
    # Loop through each level of nesting, starting with max nesting
    for level in reversed(list(set(calc_str_nesting))):
        calc_str_filt = [c for c in calc_str_w_nesting_and_pos if c[1] == level]
        
        # Figure out how many separate equations to evaluate at this level of nesting
        calc_str_filt_w_tranches = []
        tranche = 0
        for j in range(len(calc_str_filt)):
            c = calc_str_filt[j]
            if j == 0:
                pos = c[2]
                calc_str_filt_w_tranches.append([c[0], tranche, pos])
            else:
                c_prior = calc_str_filt[j-1]
                if c[2] != c_prior[2] + 1:
                    tranche+=1
                    pos = c[2]
                calc_str_filt_w_tranches.append([c[0], tranche, pos])

                    
        # Evaluate each tranche calculation
        tranche_results = []
        
        for i in range(tranche+1):
            calc_str_filt_tranche = [c[0] for c in calc_str_filt_w_tranches if c[1] == i]
            calc_str_filt_tranche_pos = min([c[2] for c in calc_str_filt_w_tranches if c[1] == i])
    
            # Repeat until whole equation is evaluated
            while len(calc_str_filt_tranche) > 1:
                calc_str_filt_tranche_new = [operator_by_order(calc_str_filt_tranche[:3])]
                # Append anything yet to be evaluated
                if len(calc_str_filt_tranche) > 3:
                    calc_str_filt_tranche_new.extend(calc_str_filt_tranche[3:])

                calc_str_filt_tranche = calc_str_filt_tranche_new

            tranche_results.append([str(calc_str_filt_tranche[0]), calc_str_filt_tranche_pos])
        
        unique_tranche_positions = list(set([c[1] for c in tranche_results]))
        
        # Create new calculation string using evaluated results
        calc_str_w_nesting_and_pos_new = []
        for i in range(len(calc_str_w_nesting_and_pos)):
            c = calc_str_w_nesting_and_pos[i]
            
            if i in unique_tranche_positions:
                tranche_result = str([c[0] for c in tranche_results if c[1] == i][0])
                calc_str_w_nesting_and_pos_new.append([tranche_result, level-1])
            elif i == 0 or c[1] < level:
                if c[1] == level:
                    calc_str_w_nesting_and_pos_new.append([calc_str_filt[0], level-1])
                if c[1] < level:
                    calc_str_w_nesting_and_pos_new.append([c[0], c[1]])
            
            
        calc_str_w_nesting_and_pos = list(zip([c[0] for c in calc_str_w_nesting_and_pos_new],
                                              [c[1] for c in calc_str_w_nesting_and_pos_new],
                                              [i for i in range(len(calc_str_w_nesting_and_pos_new))]))
            
                
    results += int(calc_str_w_nesting_and_pos[0][0])
    #break
print(results)

6811433855019


### Part 2

Similar to before, but addition must be evaluated before multiplication

In [190]:
def operator_addition_preference(calc_list):
    # Given a calc_list, applied addition first, before multiplication
    # Returns an int
    # eg. ['3','+','5','*','6'] would be parsed as 3+5 then 8*6, returning 48
    if calc_list.count('+') > 0:
        while calc_list.count('+') > 0:
            for i in range(1,len(calc_list)-1,2):
                if calc_list[i] == '+':
                    sub_calc = str(eval(''.join(calc_list[i-1:i+2])))
                    if i == 1:
                        calc_list_new = [sub_calc]
                        if len(calc_list) > 3:
                            calc_list_new.extend(calc_list[i+2:])
                    elif i < len(calc_list)-1:
                        calc_list_new = calc_list[:i-1]
                        calc_list_new.append(sub_calc)
                        calc_list_new.extend(calc_list[i+2:])
                    else:
                        calc_list_new = [sub_calc]
                        calc_list_new.extend(calc_list[i+2:])
                    break
            calc_list = copy.deepcopy(calc_list_new)
    
    return eval(''.join(calc_list))


results = 0
for i in range(len(content)):
    
    # Assign calculation string
    calc_str = content[i]
    
    # Get nesting at each position, then strip ( ) from the list
    calc_str_nesting = get_nesting(calc_str)
    calc_str_clean = [n.replace('(','').replace(')','') for n in calc_str]

    # Then, join each element of the calculation, it's nesting, and it's position
    calc_str_w_nesting_and_pos = list(zip(calc_str_clean, 
                                      calc_str_nesting, 
                                      [i for i in range(len(calc_str_clean))]))
    
    # Loop through each level of nesting, starting with max nesting
    for level in reversed(list(set(calc_str_nesting))):
        calc_str_filt = [c for c in calc_str_w_nesting_and_pos if c[1] == level]
        
        # Figure out how many separate equations to evaluate at this level of nesting
        calc_str_filt_w_tranches = []
        tranche = 0
        for j in range(len(calc_str_filt)):
            c = calc_str_filt[j]
            if j == 0:
                pos = c[2]
                calc_str_filt_w_tranches.append([c[0], tranche, pos])
            else:
                c_prior = calc_str_filt[j-1]
                if c[2] != c_prior[2] + 1:
                    tranche+=1
                    pos = c[2]
                calc_str_filt_w_tranches.append([c[0], tranche, pos])

                    
        # Evaluate each tranche calculation
        tranche_results = []
        
        for i in range(tranche+1):
            calc_str_filt_tranche = [c[0] for c in calc_str_filt_w_tranches if c[1] == i]
            calc_str_filt_tranche_pos = min([c[2] for c in calc_str_filt_w_tranches if c[1] == i])

            tranche_results.append([str(operator_addition_preference(calc_str_filt_tranche)), 
                                    calc_str_filt_tranche_pos])
        
        unique_tranche_positions = list(set([c[1] for c in tranche_results]))
        
        # Create new calculation string using evaluated results
        calc_str_w_nesting_and_pos_new = []
        for i in range(len(calc_str_w_nesting_and_pos)):
            c = calc_str_w_nesting_and_pos[i]
            
            if i in unique_tranche_positions:
                tranche_result = str([c[0] for c in tranche_results if c[1] == i][0])
                calc_str_w_nesting_and_pos_new.append([tranche_result, level-1])
            elif i == 0 or c[1] < level:
                if c[1] == level:
                    calc_str_w_nesting_and_pos_new.append([calc_str_filt[0], level-1])
                if c[1] < level:
                    calc_str_w_nesting_and_pos_new.append([c[0], c[1]])
            
            
        calc_str_w_nesting_and_pos = list(zip([c[0] for c in calc_str_w_nesting_and_pos_new],
                                              [c[1] for c in calc_str_w_nesting_and_pos_new],
                                              [i for i in range(len(calc_str_w_nesting_and_pos_new))]))
            
                
    results += int(calc_str_w_nesting_and_pos[0][0])
    #break
print(results)

129770152447927
